In [ ]:
# !git clone https://github.com/datitran/raccoon_dataset.git

**Install Tensorflow Object Detection API**

In [ ]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 52667 (delta 52), reused 59 (delta 5), pack-reused 52556
Receiving objects: 100% (52667/52667), 569.31 MiB | 35.33 MiB/s, done.
Resolving deltas: 100% (36025/36025), done.


In [ ]:
%cd /content/models/research/
!protoc object_detection/protos/*.proto --python_out=.

/content/models/research


In [ ]:
# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

Processing /content/models/research
     |████████████████████████████████| 9.0MB 13.3MB/s 
     |████████████████████████████████| 358kB 47.9MB/s 
     |████████████████████████████████| 1.1MB 41.7MB/s 
     |████████████████████████████████| 61kB 9.5MB/s 
     |████████████████████████████████| 17.7MB 213kB/s 
     |████████████████████████████████| 2.2MB 50.5MB/s 
     |████████████████████████████████| 153kB 55.9MB/s 
     |████████████████████████████████| 61kB 9.2MB/s 
     |████████████████████████████████| 829kB 45.1MB/s 
     |████████████████████████████████| 37.6MB 74kB/s 
     |████████████████████████████████| 174kB 57.6MB/s 
     |████████████████████████████████| 706kB 42.3MB/s 
     |████████████████████████████████| 1.2MB 48.3MB/s 
     |████████████████████████████████| 102kB 15.7MB/s 
     |████████████████████████████████| 645kB 43.1MB/s 
     |████████████████████████████████| 51kB 9.8MB/s 
     |████████████████████████████████| 112kB 53.6MB/s 
  Created wheel for

In [ ]:
%%bash
python /content/models/research/object_detection/builders/model_builder_tf2_test.py

2021-03-02 08:05:44.369777: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Running tests under Python 3.7.10: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model
2021-03-02 08:05:46.987223: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-02 08:05:46.988208: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-03-02 08:05:47.051617: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-03-02 08:05:47.052259: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB devi

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import tensorflow.keras as keras

import os
import io

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

print(tf.__version__)

# device = "gpu"

# if device == "tpu":
#   resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
#   tf.config.experimental_connect_to_cluster(resolver)
#   # This is the TPU initialization code that has to be at the beginning.
#   tf.tpu.experimental.initialize_tpu_system(resolver)
#   strategy = tf.distribute.experimental.TPUStrategy(resolver)
# else:
#   strategy = tf.distribute.MirroredStrategy()

2.4.1



**Generate TF Record file**

In [ ]:
"""
Usage:
  # From tensorflow/models/
  # Create train data:
  python generate_tfrecord.py --csv_input=data/train_labels.csv  --output_path=train.record
  # Create test data:
  python generate_tfrecord.py --csv_input=data/test_labels.csv  --output_path=test.record
"""
# from __future__ import division
# from __future__ import print_function
# from __future__ import absolute_import

# TO-DO replace this with label map
def class_text_to_int(row_label):
    if row_label == 'track':
        return 1
    else:
        None

def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def generate_tfrecord(output_path, image_dir, csv_input):
    writer = tf.io.TFRecordWriter(output_path)
    path = os.path.join(image_dir)
    examples = pd.read_csv(csv_input)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(), output_path)
    print('Successfully created the TFRecords: {}'.format(output_path))

In [ ]:
##Run on training data
generate_tfrecord('/content/wpafb_train.record', '/content/drive/My Drive/WPAFB_dataset/images' , '/content/drive/My Drive/WPAFB_dataset/AOI_02_tile_1024_train_short.csv')

Successfully created the TFRecords: /content/wpafb_train.record


In [ ]:
##Run on eval data
generate_tfrecord('/content/wpafb_test.record', '/content/drive/My Drive/WPAFB_dataset/images' , '/content/drive/My Drive/WPAFB_dataset/AOI_02_tile_1024_test_short.csv')

Successfully created the TFRecords: /content/wpafb_test.record



**Download a pre-trained Checkpoint model - Fine Tuning**

We use image classification checkpoints to be used in object detection model

https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_classification_zoo.md

In [ ]:
%cd /content
!ls

/content
drive  models  sample_data  wpafb_test.record  wpafb_train.record


In [ ]:
!wget http://download.tensorflow.org/models/object_detection/classification/tf2/20200710/resnet50_v1.tar.gz
!tar -xvf resnet50_v1.tar.gz

--2021-03-02 08:08:20--  http://download.tensorflow.org/models/object_detection/classification/tf2/20200710/resnet50_v1.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.20.128, 2607:f8b0:400e:c07::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.20.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87437462 (83M) [application/x-tar]
Saving to: ‘resnet50_v1.tar.gz’

resnet50_v1.tar.gz  100%[===================>]  83.39M   158MB/s    in 0.5s    

2021-03-02 08:08:21 (158 MB/s) - ‘resnet50_v1.tar.gz’ saved [87437462/87437462]

resnet50_v1/
resnet50_v1/checkpoint
resnet50_v1/resnet50.ckpt-1.data-00001-of-00002
resnet50_v1/resnet50.ckpt-1.data-00000-of-00002
resnet50_v1/resnet50.ckpt-1.index


In [ ]:
!wget http://download.tensorflow.org/models/object_detection/classification/tf2/20200710/mobilenet_v2.tar.gz
!tar -xvf mobilenet_v2.tar.gz

--2021-03-02 08:08:24--  http://download.tensorflow.org/models/object_detection/classification/tf2/20200710/mobilenet_v2.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.142.128, 2607:f8b0:400e:c07::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.142.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8404070 (8.0M) [application/x-tar]
Saving to: ‘mobilenet_v2.tar.gz’

mobilenet_v2.tar.gz 100%[===================>]   8.01M  30.8MB/s    in 0.3s    

2021-03-02 08:08:25 (30.8 MB/s) - ‘mobilenet_v2.tar.gz’ saved [8404070/8404070]

mobilenet_v2/
mobilenet_v2/mobilenet_v2.ckpt-1.index
mobilenet_v2/checkpoint
mobilenet_v2/mobilenet_v2.ckpt-1.data-00001-of-00002
mobilenet_v2/mobilenet_v2.ckpt-1.data-00000-of-00002


In [ ]:
%%bash
rm -rf wpafb_model
mkdir wpafb_model

**Training Job**

In [ ]:
%%bash
PIPELINE_CONFIG_PATH='/content/new_3_wpafb.config'
MODEL_DIR='/content/wpafb_model/'
## NUM_TRAIN_STEPS=1500
## SAMPLE_1_OF_N_EVAL_EXAMPLES=1
python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path=${PIPELINE_CONFIG_PATH} \
    --model_dir=${MODEL_DIR} \
    ## --num_train_steps=${NUM_TRAIN_STEPS}

2021-03-02 08:10:37.110326: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-03-02 08:10:39.345206: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-02 08:10:39.346083: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-03-02 08:10:39.376935: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-03-02 08:10:39.377554: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-03-02 08:10:39.377606: I tensorflow/stream_executor/platform/default/dso_loade

In [ ]:
def is_object_based_checkpoint(checkpoint_path):
  """Returns true if `checkpoint_path` points to an object-based checkpoint."""
  var_names = [var[0] for var in tf.train.list_variables(checkpoint_path)]
  return '_CHECKPOINTABLE_OBJECT_GRAPH' in var_names

In [ ]:
is_object_based_checkpoint('/content/wpafb_model')

True

**Evaluation Job - Run after atleast 1000 Train Steps**

In [18]:
%%bash
PIPELINE_CONFIG_PATH='/content/new_3_wpafb.config'
MODEL_DIR='/content/wpafb_model/'
## NUM_TRAIN_STEPS=10000
## SAMPLE_1_OF_N_EVAL_EXAMPLES=1
python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path=${PIPELINE_CONFIG_PATH} \
    --model_dir=${MODEL_DIR} \
    --checkpoint_dir=${MODEL_DIR}           ##This is passed to run ONLY EVALUATION
    ## --alsologtostderr

creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.27s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.027
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.118
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.027
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.010
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.038
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.104
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.104
 

2021-03-02 11:33:35.310805: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
W0302 11:33:37.539284 140591420110720 model_lib_v2.py:1043] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I0302 11:33:37.539481 140591420110720 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0302 11:33:37.539616 140591420110720 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0302 11:33:37.539710 140591420110720 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0302 11:33:37.539826 140591420110720 model_lib_v2.py:1058] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
2021-03-02 11:33:37.546420: I tensorflow/compiler/jit/xla

In [ ]:
## !kill 26692
%tensorboard --logdir "/content/wpafb_model"